<a href="https://colab.research.google.com/github/alvarofpinheiro/random_forest/blob/main/Simulacao1_Cenario1_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Random Forest - método geral de florestas de decisão aleatória foi proposto pela primeira vez por Tin Kam Ho em 1995. Ho estabeleceu que as florestas de árvores que se dividem com hiperplanos oblíquos podem ganhar precisão à medida que crescem sem sofrer de overtraining, desde que as florestas sejam restritas aleatoriamente para serem sensíveis apenas a dimensões de recursos selecionados.

Ho, T.K. Random Decision Forests. Proceedings of the 3rd International Conference on Document Analysis and Recognition, Montreal, QC, 14–16. pp. 278–282. https://doi.org/10.1109/ICDAR.1995.598994, 1995.

Explicação do algoritmo RF: https://youtu.be/ByUoVg4QI84

In [ ]:
#instalar biblioteca Orange Canvas
!pip install Orange3

In [ ]:
#importar biblioteca Orange Canvas
import Orange

In [ ]:
#importar dados do disco local
from google.colab import files  
files.upload()

In [ ]:
#instanciar objeto de dados com base no caminho gerado com a importação do arquivo
dados = Orange.data.Table("/content/Simulacao1.csv")

In [ ]:
#exibir os primeiros 5 registros para uma rápida análise da importação dos dados
for d in dados[:5]:
  print(d)

In [ ]:
#explorar os metadados da base de dados importada
qtde_campos = len(dados.domain.attributes)
qtde_cont = sum(1 for a in dados.domain.attributes if a.is_continuous)
qtde_disc = sum(1 for a in dados.domain.attributes if a.is_discrete)
print("%d metadados: %d continuos, %d discretos" % (qtde_campos, qtde_cont, qtde_disc))
print("Nome dos metadados:", ", ".join(dados.domain.attributes[i].name for i in range(qtde_campos)),)

In [ ]:
#explorar domínios dos atributos (campos da base de dados)
dados.domain.attributes

In [ ]:
#explorar as classes da base de dados
dados.domain.class_var

In [ ]:
#explorar dados (quantidade de campos e registros da base de dados)
print("Campos:", ", ".join(c.name for c in dados.domain.attributes))
print("Registros:", len(dados))
print("Valor do registro 1 da coluna 1:", dados[0][0])
print("Valor do registro 2 da coluna 2:", dados[1][1])

In [ ]:
#criar amostra de dados, sendo 70% para treinamento e teste e 30% para avaliação
qtde100 = len(dados)
qtde70 = len(dados) * 70 / 100
qtde30 = len(dados) * 30 / 100
print("Qtde 100%:", qtde100)
print("Qtde  70%:", qtde70)
print("Qtde  30%:", qtde30)
amostra = Orange.data.Table(dados.domain, [d for d in dados if d.row_index < qtde70])
print("Registros:", len(dados))
print("Registros:", len(amostra))

**Hiperparâmetros do Random Forest**

n_estimators (int, optional (default = 10)): define quantidade de árvores na floresta.

criterion = 'gini': técnica para escolher a melhor variável para compor o nó raiz.

max_depth (int, optional (default = 1024)): define a profundidade máxima da árvore.

min_samples_split (float): número mínimo de instâncias de dados que são divididas em subgrupos.

min_samples_leaf (float): número mínimo de instâncias de dados em uma folha.

min_weight_fraction_leaf (float): peso mínimo.

max_features='auto': máximo de características.

max_leaf_nodes=None: máximo de nós folha.

bootstrap=True: técnica de inicialização (obter dados de amostragem).

oob_score=False: se irá realizar calculo da pontuação.

n_jobs=1: número de repetição do trabalho.

random_state=None: técnica de aleatoriedade.

verbose=0: se o processamento será impresso durante a execução.

class_weight=None: peso da classe.

preprocessors=None: é usado o pré-processamento padrão quando nenhum outro pré-processador é fornecido. Ele os executa na seguinte ordem: remove instâncias com valores de destino desconhecidos; continua variáveis categóricas (com codificação one-hot); remove colunas vazias; imputa valores ausentes com valores médios.

In [ ]:
#inicializar a instância com os seus hiperparâmetros para a técnica Random Forest (RF) Cenário1 - recomendado pelo Kernel
rf = Orange.classification.RandomForestLearner(
    n_estimators=10, 
    criterion='gini', 
    max_depth=None, 
    min_samples_split=2, 
    min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, 
    max_features='auto', 
    max_leaf_nodes=None, 
    bootstrap=True, 
    oob_score=False, 
    n_jobs=1, 
    random_state=None, 
    verbose=0, 
    class_weight=None, 
    preprocessors=None)

In [ ]:
#ligar técnica RF com os dados da simulação
dados_rf = rf(dados)

In [ ]:
#treinar e testar técnica RF com os dados com 5 partes (folders)
avalia_rf = Orange.evaluation.CrossValidation(dados, [rf], k=5)

In [ ]:
#avaliar os indicadores para o RF com base nas métricas de classificação
print("Acurácia:  %.3f" % Orange.evaluation.scoring.CA(avalia_rf)[0])
print("Precisão:  %.3f" % Orange.evaluation.scoring.Precision(avalia_rf)[0])
print("Revocação: %.3f" % Orange.evaluation.scoring.Recall(avalia_rf)[0])
print("F1:        %.3f" % Orange.evaluation.scoring.F1(avalia_rf)[0])
print("ROC:       %.3f" % Orange.evaluation.scoring.AUC(avalia_rf)[0])

In [ ]:
#inicializar a instância com os seus hiperparâmetros para a técnica K-Nearest Neighbors (KNN) Cenário2 - primeira escolha aleatória para avaliar a recomendação do Kernel
knn = Orange.classification.KNNLearner(
    n_neighbors=5, 
    metric='euclidean', 
    weights='distance', 
    algorithm='auto', 
    metric_params=None, 
    preprocessors=None)

In [ ]:
#inicializar a instância com os seus hiperparâmetros para a técnica Support Vector Machine (SVM) Cenário3 - segunda escolha aleatória para avaliar a recomendação do Kernel
svm = Orange.classification.SVMLearner(
    C=1.0, 
    kernel='rbf', 
    degree=3, 
    gamma='auto', 
    coef0=0.0, 
    shrinking=True, 
    probability=False, 
    tol=0.001, 
    cache_size=2000, 
    max_iter=-1, 
    preprocessors=None)

In [ ]:
#validar o aprendizado para as 3 técnicas
aprendizado = [rf, knn, svm]
avaliacao = Orange.evaluation.CrossValidation(dados, aprendizado, k=5)

In [ ]:
#imprimir os indicadores para as 3 técnicas
print(" " * 10 + " | ".join("%-4s" % learner.name for learner in aprendizado))
print("Acurácia  %s" % " | ".join("%.2f" % s for s in Orange.evaluation.CA(avaliacao)))
print("Precisão  %s" % " | ".join("%.2f" % s for s in Orange.evaluation.Precision(avaliacao)))
print("Revocação %s" % " | ".join("%.2f" % s for s in Orange.evaluation.Recall(avaliacao)))
print("F1        %s" % " | ".join("%.2f" % s for s in Orange.evaluation.F1(avaliacao)))
print("ROC       %s" % " | ".join("%.2f" % s for s in Orange.evaluation.AUC(avaliacao)))